# DD-Pose getting started
This jupyter notebook shows you how to access the raw data and annotations of the DD-Pose dataset

In [ ]:
from dd_pose.dataset import Dataset
from dd_pose.dataset_item import DatasetItem
from dd_pose.image_decorator import ImageDecorator
from dd_pose.jupyter_helpers import showimage
from dd_pose.evaluation_helpers import T_headfrontal_camdriver, T_camdriver_headfrontal
from dd_pose.visualization_helpers import get_dashboard

import transformations as tr

import numpy as np

A `Dataset` contains `dataset item dictionaries`.  
You can choose between splits `'trainval'`, `'test'` and `'all'`:
* `trainval`: training and validation split. Raw data and head pose measurements
* `test`: held-out test split. No head pose measurements
* `all`: union of the two above

In [ ]:
d = Dataset(split='all')
len(d)

`d.get_dataset_items()` yeilds a generator for all `dataset item dictionaries` in a dataset.  
A `dataset_item dictionary` is represented by a `subject` (int), a `scenario` (int) and a `humanhash` (str).  
The `humanhash` is there to disambiguate multiple `scenario`s of the same type.

In [ ]:
next(d.get_dataset_items())

You can also get a `dataset item dictionary` by providing `subject`, `scenario` and `humanhash` directly.  
The file `resources/dataset-items-trainval.txt` covers the existing dataset items of the `trainval` split.

In [ ]:
!head resources/dataset-items-trainval.txt

In [ ]:
di_dict = d.get(subject_id=1, scenario_id=3, humanhash='sodium-finch-fillet-spring') # trainval
# di_dict = d.get(subject_id=6, scenario_id=0, humanhash='quebec-aspen-washington-social') # test
di_dict

## Access data

A `DatasetItem` object encapsulates all data

In [ ]:
di = DatasetItem(di_dict)

A measurement in a `DatasetItem` is indexed by a timestamp.  
`di.get_stamps()` gets all timestamps as long integers.

In [ ]:
stamps = di.get_stamps()
len(stamps)

In [ ]:
stamps[0:5]

Choose arbitrary stamp and print data for it

In [ ]:
stamp = stamps[153]

### Get gps information

In [ ]:
di.get_gps(stamp)

### Heading above ground

In [ ]:
di.get_heading(stamp)

### Get the image of the left driver cam
... and convert from 16bit depth to 8bit depth by shifting 8 bits to the right

In [ ]:
img, pcm = di.get_img_driver_left(stamp, shift=True)
img.shape, img.dtype

### pcm represents the associated pinhole camera model

In [ ]:
pcm.P

### You can project 3d points onto the image plane

In [ ]:
pcm.project3dToPixel((0, 0, 1))

### This is the image of the left driver camera

In [ ]:
showimage(img)

### Get the docu cam image

In [ ]:
img_docu, pcm_docu = di.get_img_docu(stamp)
img_docu.shape, img_docu.dtype

In [ ]:
showimage(img_docu)

### Occlusion state of face (see paper)

In [ ]:
di.get_occlusion_state(stamp)

### Steering wheel angle and acceleration

In [ ]:
di.get_stw_angle(stamp)

### Transformations
Transformations are given in homogeneous coordinates.  
The terminology is:  
`point_A = T_A_B * point_B`

`T_A_B` is a homogeneous 4x4 matrix which transforms a `point_B` from frame `B` to a `point_A` in frame `A`.  
Points are homogeneous 4 element column vectors `(x, y, z, 1.0)`.  

### Static homogeneous transformation from body frame (car) to camdriver (the optical frame of the left driver camera)

In [ ]:
di.get_T_camdriver_body()

### Static homogeneous transformation from camdocu optical frame to camdriver optical frame

In [ ]:
di.get_T_camdriver_camdocu()

### Static homogeneous transformation from gps frame to camdriver optical frame

In [ ]:
di.get_T_camdriver_gps()

### Head pose: homogeneous transformation from head frame to camdriver optical frame

In [ ]:
T_camdriver_head = di.get_T_camdriver_head(stamp)
T_camdriver_head

### Draw head pose into camdriver image

In [ ]:
img, pcm = di.get_img_driver_left(stamp, shift=True)
img_bgr = np.dstack((img, img, img))

image_decorator = ImageDecorator(img_bgr, pcm)
if T_camdriver_head is not None:
    image_decorator.draw_axis(T_camdriver_head)
else:
    image_decorator.draw_text("no T_camdriver_head")
showimage(img_bgr)

### Draw head pose into camdocu image

In [ ]:
img_docu, pcm_docu = di.get_img_docu(stamp)
image_decorator = ImageDecorator(img_docu, pcm_docu)

# Get transformation from head into camdocu frame by "chaining"
# Note how the 'camdriver' cancels out by multiplication
T_camdocu_camdriver = np.linalg.inv(di.get_T_camdriver_camdocu())
if T_camdriver_head is not None:
    T_camdocu_head = np.dot(T_camdocu_camdriver, T_camdriver_head)
    image_decorator.draw_axis(T_camdocu_head)
else:
    image_decorator.draw_text("No T_camdriver_head")

# Also draw camdriver frame into image
image_decorator.draw_axis(T_camdocu_camdriver)

showimage(img_docu)

### Angular representation of head pose
There are many angular representations.  
You get get a conventional representation by 'static axis rotation' towards frontally looking head, 
i.e. `roll = pitch = yaw = 0` represents a head looking frontally towards the camera


In [ ]:
if T_camdriver_head is not None:
    T_headfrontal_head = np.dot(T_headfrontal_camdriver, T_camdriver_head)
    roll, pitch, yaw = tr.euler_from_matrix(T_headfrontal_head, 'sxyz')
    roll, pitch, yaw # in rad

### Visualize `T_camdriver_headfrontal`
Draw `headfrontal` frame into camdocu image.  
`x` points inside the camera

In [ ]:
img_docu, pcm_docu = di.get_img_docu(stamp)
image_decorator = ImageDecorator(img_docu, pcm_docu)
T_camdocu_camdriver = np.linalg.inv(di.get_T_camdriver_camdocu())
T_camdocu_headfrontal = np.dot(T_camdocu_camdriver, T_camdriver_headfrontal)
image_decorator.draw_axis(T_camdocu_headfrontal)
showimage(img_docu)

### Use all-in-one function `get_dashboard`
Shows all information in one image

In [ ]:
img_dashboard = get_dashboard(di, stamp)
showimage(img_dashboard)